In [1]:
!pip install plotly
!pip install openpyxl
!pip install -U scikit-learn
!pip install statsmodels
!pip install catboost

In [2]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_csv('data/dataset_hackaton_ksg__v2__23062023__1710_GMT3.csv',sep=';',parse_dates=True,index_col='date_report',low_memory=False)
attr = pd.read_csv('data/data_mgz_attributes.csv',sep=';',parse_dates=True,index_col='date_report')
name = pd.read_excel ('data/names.xlsx',index_col='Кодзадачи')
#name = name.to_dict()



# attr = attr.rename(columns={'Код ДС': 'obj_key'})
# completed = dataset['obj_key'][dataset['ПроцентЗавершенияЗадачи'] == 100].unique()
# f10 = dataset['obj_key'].value_counts().head(10).index.to_list()
# dataset = dataset[dataset['obj_key'].isin(f10)]
# dataset = dataset[dataset['Кодзадачи'].isin(name.index)]



In [4]:
f10 = dataset['obj_key'].value_counts().head(15).index.to_list()
dataset = dataset[dataset['obj_key'].isin(f10)]

In [5]:
dataset['ДатаНачалаЗадачи'] = pd.to_datetime(dataset['ДатаНачалаЗадачи'])
dataset['ДатаОкончанияЗадачи'] = pd.to_datetime(dataset['ДатаОкончанияЗадачи'])
dataset['ДатаначалаБП0'] = pd.to_datetime(dataset['ДатаначалаБП0'])
dataset['ДатаокончанияБП0'] = pd.to_datetime(dataset['ДатаокончанияБП0'])

In [6]:
dataset ['stime_delta'] = (dataset['ДатаначалаБП0'] - dataset['ДатаНачалаЗадачи']).dt.days 
dataset ['etime_delta'] = (dataset['ДатаокончанияБП0'] - dataset['ДатаОкончанияЗадачи']).dt.days 
dataset ['wtime_delta'] = ((dataset['ДатаОкончанияЗадачи'] - dataset['ДатаНачалаЗадачи']) - (dataset['ДатаокончанияБП0'] - dataset['ДатаначалаБП0'])).dt.days

In [7]:
dataset

,Unnamed: 0,№ п/п,obj_prg,obj_subprg,obj_key,Кодзадачи,НазваниеЗадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,ДатаначалаБП0,ДатаокончанияБП0,Статуспоэкспертизе,Экспертиза,stime_delta,etime_delta,wtime_delta
date_report,,,,,,,,,,,,,,,,,
2023-01-17,2,47,Образование,Дошкольные учреждения,022-0513,1,Предпроектные работы,0.0,2023-01-23,2023-09-29,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2023-01-17,6,43,Образование,Дошкольные учреждения,021-0548,1,Предпроектные работы,100.0,2019-03-15,2022-06-01,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2023-01-17,16,61,Образование,Общеобразовательные учреждения,019-0676,1,Предпроектные работы,100.0,2021-02-01,2022-09-15,2021-02-01,2022-07-31,NaN,NaN,0.0,-46.0,46.0
2023-01-17,20,55,Образование,Общеобразовательные учреждения,019-0429,1,Предпроектные работы,100.0,2019-08-12,2022-04-04,2019-08-12,2022-04-04,NaN,NaN,0.0,0.0,0.0
2023-01-17,22,54,Образование,Общеобразовательные учреждения,019-0428,1,Предпроектные работы,100.0,2020-09-07,2022-01-20,2020-09-07,2022-01-20,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-29,63470,18,Здравоохранение,Поликлиники,019-0466,9.6.2,КC-11 оформлена (АКТ приёмки завершенного стро...,0.0,2023-07-31,2023-07-31,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2023-05-29,63473,6,Здравоохранение,ССК,019-0478,9.6.2,КC-11 оформлена,0.0,2023-06-09,2023-06-09,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2023-05-29,63477,7,Здравоохранение,ССК,019-0481,9.6.2,КC-11 оформлена,0.0,2024-02-16,2024-02-16,NaT,NaT,NaN,NaN,NaN,NaN,NaN


In [8]:
# Вычисление среднего значения и стандартного отклонения
mean = np.mean(dataset ['stime_delta'])
std = np.std(dataset ['stime_delta'])

lower_threshold = mean - 3 * std
upper_threshold = mean + 3 * std
dataset = dataset[(dataset['stime_delta']>lower_threshold) & (dataset['stime_delta']<upper_threshold)]

In [9]:
mean = np.mean(dataset ['etime_delta'])
std = np.std(dataset ['etime_delta'])

lower_threshold = mean - 3 * std
upper_threshold = mean + 3 * std
dataset = dataset[(dataset['etime_delta']>lower_threshold) & (dataset['etime_delta']<upper_threshold)]

In [10]:
mean = np.mean(dataset ['wtime_delta'])
std = np.std(dataset ['wtime_delta'])

lower_threshold = mean - 3 * std
upper_threshold = mean + 3 * std
dataset = dataset[(dataset['wtime_delta']>lower_threshold) & (dataset['wtime_delta']<upper_threshold)]

In [11]:
upper_threshold

199.828154539215

In [12]:
#dataset ['time_delta'] = (dataset['ДатаНачалаЗадачи'] - dataset['ДатаОкончанияЗадачи']).dt.days 
dataset = dataset.drop(['Unnamed: 0','№ п/п','НазваниеЗадачи','ДатаНачалаЗадачи',
       'ДатаОкончанияЗадачи', 'ДатаначалаБП0', 'ДатаокончанияБП0'],axis=1)
dataset = dataset.dropna(subset=['stime_delta','etime_delta','wtime_delta'])

In [13]:
cat_features = ['obj_prg','obj_subprg','obj_key','Кодзадачи','Экспертиза','Статуспоэкспертизе']

In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 218995 entries, 2023-01-17 to 2023-05-29
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   obj_prg                  218995 non-null  object 
 1   obj_subprg               218995 non-null  object 
 2   obj_key                  218995 non-null  object 
 3   Кодзадачи                218995 non-null  object 
 4   ПроцентЗавершенияЗадачи  218995 non-null  float64
 5   Статуспоэкспертизе       4213 non-null    float64
 6   Экспертиза               709 non-null     object 
 7   stime_delta              218995 non-null  float64
 8   etime_delta              218995 non-null  float64
 9   wtime_delta              218995 non-null  float64
dtypes: float64(5), object(5)
memory usage: 18.4+ MB


In [15]:
dataset = pd.get_dummies(dataset, columns=cat_features)

In [16]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

train,test = train_test_split(dataset,
                            test_size=1 - train_ratio,
                            random_state=12345,
                            shuffle=False
                            )
train = train.dropna()
features_train = train.drop(['stime_delta','etime_delta','wtime_delta'],axis=1)
target_train = train[['stime_delta','etime_delta','wtime_delta']]
features_test = test.drop(['stime_delta','etime_delta','wtime_delta'],axis=1)
target_test = test[['stime_delta','etime_delta','wtime_delta']]


features_valid, features_test, target_valid, target_test = train_test_split(features_test,
                                                                        target_test,
                                                                        test_size=test_ratio/(test_ratio + validation_ratio),
                                                                        random_state=12345,
                                                                        shuffle=False
                                                                       )

In [17]:
features_train

,ПроцентЗавершенияЗадачи,obj_prg_Здравоохранение,obj_prg_Образование,obj_subprg_Дошкольные учреждения,obj_subprg_Общеобразовательные учреждения,obj_subprg_Поликлиники,obj_subprg_ССК,obj_key_019-0428,obj_key_019-0429,obj_key_019-0465,...,Кодзадачи_9.6.1,Кодзадачи_9.6.2,Экспертиза_1С,Экспертиза_1Т,Экспертиза_1ТС,Экспертиза_2С,Экспертиза_2Т,Экспертиза_2ТС,Статуспоэкспертизе_1.0,Статуспоэкспертизе_2.0
date_report,,,,,,,,,,,,,,,,,,,,,
2023-01-17,100.0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-17,100.0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-17,100.0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-17,100.0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-17,100.0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-10,16.0,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2023-05-10,68.0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-05-10,60.0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
model = LinearRegression()
model.fit(features_train,target_train)
predictions = model.predict(features_valid)
rmse = np.sqrt(mean_squared_error(target_valid,predictions))
print(rmse)

61.96407305711243


In [22]:
model = CatBoostRegressor(random_state=12345)
parametrs = {'learning_rate': np.arange(0.1,0.35,0.05)}

grid = GridSearchCV(model, parametrs,n_jobs=-1,cv=TimeSeriesSplit(n_splits=2))
grid.fit(features_train,target_train)
predictions = grid.predict(features_valid)
rmse =np.sqrt(mean_squared_error(target_valid,predictions))
print(rmse)
print(grid.best_params_)

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/catboost/core.py", line 5734, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "/usr/local/lib/python3.8/dist-packages/catboost/core.py", line 2357, in _fit
    self._train(
  File "/usr/local/lib/python3.8/dist-packages/catboost/core.py", line 1761, in _train
    self._object._train(train_pool, test_pool, params, allow_clear_pool, init

In [ ]:
# attr = attr.rename(columns={'Код ДС': 'obj_key'})

In [ ]:
# completed = dataset['obj_key'][dataset['ПроцентЗавершенияЗадачи'] == 100].unique()
# f10 = dataset['obj_key'].value_counts().head(2).index.to_list()
# dataset = dataset[dataset['obj_key'].isin(f10)]
# df = pd.merge(dataset, attr, on='obj_key', how='left')
# dataset = dataset[dataset['obj_key'] == '022-0527']
# dataset[dataset['ПроцентЗавершенияЗадачи'] ==100]